<a href="https://colab.research.google.com/github/danielsineus/Financial-Analysis/blob/master/factor_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This exercise has been thought to help anyone who like to create a **basic, quantitative multi-factor investment strategy**. The sample universe of stocks has been in csv format retrieved from Ycharts.The idea and the methods came from a class of Portfolio management taken with a brilliant Professor, Mr.  Matthews [link text](https://www.linkedin.com/search/results/all/?keywords=michael%20j.%20matthews%2C%20cfa&origin=RICH_QUERY_SUGGESTION&position=1&searchId=2cbc037d-52d2-43f3-9557-edecb8f78b83&sid=fYg)



# **INVESTMENT DECISION BASED ON FACTOR ANALYSIS**

---
The purpose of this document is to bring a factor analysis toward a portfolio by using a great deal of securities reflecting the S&P500. Anyone will be able to see all the steps from begining to the end. One thing that you will need to expect is that there will not be theories about Quantitative Portfolio Management. You will discover more the routes taken to get the results.


---



In [29]:
#upload the file
import pandas as pd
from pandas import Series, DataFrame


In [2]:
from google.colab import files
uploaded=files.upload()

Saving MultifactorDaniel.csv to MultifactorDaniel.csv


In [30]:
factor=pd.read_csv("MultifactorDaniel.csv")
factor1=pd.DataFrame(factor)
factor1.drop(["Name", "Industry"], inplace=True, axis=1) # Delete the collumns Name and Industry
factor1.head()#Check the result

,Symbol,Sector,PE Ratio,Profit Margin,1 Year Total Returns,Std_Monthly Returns,Market Cap
0,A,Healthcare,48.7469,0.161185,0.800574,0.121396,4.722171e+04
1,AAL,Industrials,NaN,-0.310804,0.817332,0.188876,1.401746e+04
2,AAP,Consumer Cyclical,22.2937,0.057326,0.488989,0.090504,1.310415e+04
3,AAPL,Technology,27.9159,0.250038,0.366343,0.146120,2.358028e+06
4,ABBV,Healthcare,29.4043,0.124011,0.308125,0.075381,1.927812e+05


Above, you can notice 6 factors. The following factors are 

*   **Value Factors**: Price to Earnings ratio;
*   **Quality Factors**: Return on Equity and Profit margin;
*   **Momentum Factors**: Past 12 month return;
*   **Volatility** : Standard deviation;
*   **Size** : Market Capitalization



In [31]:
print(factor1.isnull().sum()) #to compute the sum of empty cells per variables

Symbol                   0
Sector                   0
PE Ratio                39
Profit Margin            3
1 Year Total Returns     1
Std_Monthly Returns      0
Market Cap               0
dtype: int64


The variable PE ratio alone has 39 missing values. We infer that there are many missing values, so we need to deal with the missing values. we can't afford to drop the empty cells. We resort to the stategy of replacing the empty cells by the median.

In [5]:
# Replace missing values. we could've dropped the missing values by using dropna() because they are too much in the column P/E ratio
print(factor1.isnull().sum().sum())

43


In [6]:
#use fillna()
medi=factor1["PE Ratio"].median()
medi


26.0248

In [32]:
#Replace missing values with the median number
factor1["PE Ratio"].fillna(medi,inplace=True)
#Check to see if there is any missing value in the column **PE Ratio**
factor1["PE Ratio"].isnull().values.any()
factor1.isnull().sum()

Symbol                  0
Sector                  0
PE Ratio                0
Profit Margin           3
1 Year Total Returns    1
Std_Monthly Returns     0
Market Cap              0
dtype: int64

In [8]:
factor1.dropna(how="any")

,Symbol,Sector,PE Ratio,Profit Margin,1 Year Total Returns,Std_Monthly Returns,Market Cap
0,A,Healthcare,48.7469,0.161185,0.800574,0.121396,4.722171e+04
1,AAL,Industrials,26.0248,-0.310804,0.817332,0.188876,1.401746e+04
2,AAP,Consumer Cyclical,22.2937,0.057326,0.488989,0.090504,1.310415e+04
3,AAPL,Technology,27.9159,0.250038,0.366343,0.146120,2.358028e+06
4,ABBV,Healthcare,29.4043,0.124011,0.308125,0.075381,1.927812e+05
...,...,...,...,...,...,...,...
500,YUM,Consumer Cyclical,28.3578,0.212136,0.397388,0.078029,3.655375e+04
501,ZBH,Healthcare,34.2821,0.116138,0.151918,0.077708,3.121539e+04
502,ZBRA,Technology,36.1117,0.147560,1.231938,0.109852,2.728786e+04
503,ZION,Financial Services,9.5714,0.390975,1.195801,0.101131,1.031579e+04


We still have a few missing values. Nothing to impede us analysis. We find it correct to drop the few missing values left. It would not impact our analysis substantially. 

In [33]:
factor1.head()

,Symbol,Sector,PE Ratio,Profit Margin,1 Year Total Returns,Std_Monthly Returns,Market Cap
0,A,Healthcare,48.7469,0.161185,0.800574,0.121396,4.722171e+04
1,AAL,Industrials,26.0248,-0.310804,0.817332,0.188876,1.401746e+04
2,AAP,Consumer Cyclical,22.2937,0.057326,0.488989,0.090504,1.310415e+04
3,AAPL,Technology,27.9159,0.250038,0.366343,0.146120,2.358028e+06
4,ABBV,Healthcare,29.4043,0.124011,0.308125,0.075381,1.927812e+05


In [10]:
# use the factor method to rank
factor1["rank_Profit"]=factor1["Profit Margin"].rank(ascending=False)
factor1["rank_MarketCap"]=factor1["Market Cap"].rank(ascending=False)
factor1["rank_riskvalue"]=factor1["Std_Monthly Returns"].rank(ascending=False)
factor1["rank_returns"]=factor1["1 Year Total Returns"].rank(ascending=False)
factor1["rank_PE"]=factor1["PE Ratio"].rank(ascending=True)


In [34]:
factor1.head()
fact=factor1.index
fact1=len(fact)# count the number of rows in a Pandas dataframe.
fact1# the higher the PE Ratio the riskier the firm might be. this is the reason why we use the ascending=True, it means the lower PE ratio the better


505

In [12]:
# Computer the average of the rank
factor1["average_rank"]=factor1[["rank_Profit", "rank_MarketCap", "rank_riskvalue", "rank_returns", "rank_PE"]].mean(axis=1)
factor1.head()

,Symbol,Sector,PE Ratio,Profit Margin,1 Year Total Returns,Std_Monthly Returns,Market Cap,rank_Profit,rank_MarketCap,rank_riskvalue,rank_returns,rank_PE,average_rank
0,A,Healthcare,48.7469,0.161185,0.800574,0.121396,4.722171e+04,181.0,162.0,114.0,102.0,421.0,196.0
1,AAL,Industrials,26.0248,-0.310804,0.817332,0.188876,1.401746e+04,493.0,423.0,16.0,99.0,253.0,256.8
2,AAP,Consumer Cyclical,22.2937,0.057326,0.488989,0.090504,1.310415e+04,403.0,440.0,285.0,229.0,185.0,308.4
3,AAPL,Technology,27.9159,0.250038,0.366343,0.146120,2.358028e+06,87.0,1.0,61.0,299.0,288.0,147.2
4,ABBV,Healthcare,29.4043,0.124011,0.308125,0.075381,1.927812e+05,254.0,45.0,406.0,332.0,310.0,269.4


In [133]:
factor1["rank"]=factor1["average_rank"].rank(ascending=False)
factor1.sort_values(by="average_rank").head(10)


KeyError: ignored

In [36]:
#Convert the data into a spreadsheet in case you would like to share it with non-python users
factor1.to_csv("Mathew1.csv", index=False)


# Investment decision
We invest 30% in stocks in the given universe. TO determine the neutral weight. it is 30%  * 502

In [14]:
stocksel=round(.28*505,ndigits=0)#Let's select the first 152 stocks
stocksel
weight=round(1/stocksel, ndigits=3)
weight

0.007

each stock will have a weight of 0.007.


In [25]:
securities=factor1["Symbol"].iloc[:150]
securities

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
145     DRI
146     DTE
147     DUK
148     DVA
149     DVN
Name: Symbol, Length: 150, dtype: object

##FACTOR MODEL BY SECTOR
To proceed with the factor model, we are going to group the data by sectors particularly when doing the factor ranking. 

In [15]:

factor2=factor1
factor2=factor1[["Symbol", "Sector", "PE Ratio", "Profit Margin", "1 Year Total Returns", "Std_Monthly Returns", "Market Cap"]]
factor2.head(10)

,Symbol,Sector,PE Ratio,Profit Margin,1 Year Total Returns,Std_Monthly Returns,Market Cap
0,A,Healthcare,48.7469,0.161185,0.800574,0.121396,4.722171e+04
1,AAL,Industrials,26.0248,-0.310804,0.817332,0.188876,1.401746e+04
2,AAP,Consumer Cyclical,22.2937,0.057326,0.488989,0.090504,1.310415e+04
3,AAPL,Technology,27.9159,0.250038,0.366343,0.146120,2.358028e+06
4,ABBV,Healthcare,29.4043,0.124011,0.308125,0.075381,1.927812e+05
5,ABC,Healthcare,26.0248,-0.018323,0.312442,0.094276,2.489497e+04
6,ABMD,Healthcare,96.2292,0.165101,0.334857,0.178700,1.467278e+04
7,ABT,Healthcare,33.0169,0.158477,0.250920,0.067961,2.077919e+05
8,ACN,Technology,35.5038,0.116889,0.603011,0.077423,2.054974e+05
9,ADBE,Technology,47.7643,0.386714,0.331516,0.134916,2.747602e+05


In [37]:
factor2.set_index("Sector")
factor2.head(30)
factor2.tail(30)
dat=factor2.sort_values("Sector", ascending=True)

In [ ]:
dat.head(10)

,Symbol,Sector,PE Ratio,Profit Margin,1 Year Total Returns,Std_Monthly Returns,Market Cap
340,NUE,Basic Materials,9.6971,0.117009,1.313789,0.101540,28679.2679
154,ECL,Basic Materials,58.6359,0.086625,0.160025,0.073026,61122.5303
409,SHW,Basic Materials,35.7641,0.110501,0.286593,0.078556,75051.7446
306,MLM,Basic Materials,28.8047,0.158085,0.599970,0.088028,22082.1880
41,APD,Basic Materials,28.9573,0.201540,-0.086945,0.076976,56985.9147
469,VMC,Basic Materials,34.4502,0.136388,0.381523,0.079334,22945.3852
236,IFF,Basic Materials,115.6557,0.016821,0.165208,0.071176,35913.4352
22,ALB,Basic Materials,35.4765,0.217767,1.742515,0.098211,25599.1636
289,LYB,Basic Materials,7.8614,0.115709,0.382934,0.098322,32056.6108
90,CF,Basic Materials,28.9763,0.099455,0.837812,0.119988,13151.2140


In [38]:

factor2["rank1_PE"]=factor2.groupby("Sector")["PE Ratio"].rank(ascending=True)
factor2["rank1_Profit"]=factor2.groupby("Sector")["Profit Margin"].rank(ascending=False)
factor2["rank1_Return"]=factor2.groupby("Sector")["1 Year Total Returns"].rank(ascending=False)
factor2["rank1_risk"]=factor2.groupby("Sector")["Std_Monthly Returns"].rank(ascending=False)
factor2["rank1_cap"]=factor2.groupby("Sector")["Market Cap"].rank(ascending=False)

In [92]:
df.head()

,Symbol,Sector,PE Ratio,Profit Margin,1 Year Total Returns,Std_Monthly Returns,Market Cap,rank1_PE,rank1_Profit,rank1_Return,rank1_risk,rank1_cap
340,NUE,Basic Materials,9.6971,0.117009,1.313789,0.101540,28679.2679,4.0,10.0,2.0,7.0,13.0
154,ECL,Basic Materials,58.6359,0.086625,0.160025,0.073026,61122.5303,20.0,18.0,18.0,19.0,3.0
409,SHW,Basic Materials,35.7641,0.110501,0.286593,0.078556,75051.7446,17.0,14.0,13.0,16.0,2.0
306,MLM,Basic Materials,28.8047,0.158085,0.599970,0.088028,22082.1880,12.0,6.0,6.0,13.0,16.0
41,APD,Basic Materials,28.9573,0.201540,-0.086945,0.076976,56985.9147,13.0,5.0,20.0,18.0,4.0


In [40]:
df=pd.DataFrame(factor2.sort_values("Sector"))
#Convert the data into a spreadsheet in case you would like to share it with non-python users
df.to_csv("Mathew2.csv", index=False)
# number of stock per sectors to invest in
number=df.groupby(["Sector"])["Symbol"].count()
number.tail(13)
dframe=pd.DataFrame(number, index=None)
dframe


,Symbol
Sector,
Basic Materials,21
Communication Services,27
Consumer Cyclical,66
Consumer Defensive,35
Energy,21
Financial Services,69
Healthcare,65
Industrials,73
Real Estate,29


We are going to keep the investment decision the same way. We will invest 30% in stocks in the given universe given all the sectors. To
 determine the neutral weight. it is 30% * by the quatity of stocks per sector



In [39]:
dframe["Stock_per_sector"]=round(dframe["Symbol"]*0.3, ndigits=None)
dframe

,Symbol,Stock_per_sector
Sector,,
Basic Materials,21,6.0
Communication Services,27,8.0
Consumer Cyclical,66,20.0
Consumer Defensive,35,10.0
Energy,21,6.0
Financial Services,69,21.0
Healthcare,65,20.0
Industrials,73,22.0
Real Estate,29,9.0


In [116]:

Symol1=df.loc[df["Sector"]=="Basic Materials","Symbol"].head(6)
Symol2=df.loc[df["Sector"]=="Comunications Services","Symbol"].head(8)
Symol3=df.loc[df["Sector"]=="Consumer Cyclical","Symbol"].head(20)   
Symol4=df.loc[df["Sector"]=="Consumer Defensive","Symbol"].head(10) 
Symol5=df.loc[df["Sector"]=="Energy","Symbol"].head(6)
Symol6=df.loc[df["Sector"]=="Financial Services","Symbol"].head(21)
Symol7=df.loc[df["Sector"]=="Healthcare","Symbol"].head(20)
Symol8=df.loc[df["Sector"]=="Industrials","Symbol"].head(22)
Symol9=df.loc[df["Sector"]=="Real Estate","Symbol"].head(9)
Symol10=df.loc[df["Sector"]=="Technology","Symbol"].head(21)
Symol11=df.loc[df["Sector"]=="Utilities","Symbol"].head(8)

The investable universe when considering the top stock per sector. Please see the list of the 143 securities. in case you want to see 140 securities. use the function result.head(143)

In [128]:
frames=[Symol1, Symol2,Symol3, Symol4,Symol5, Symol6, Symol7,Symol8,Symol9,Symol10,Symol11]
result=pd.concat(frames)
result.head(30)


340     NUE
154     ECL
409     SHW
306     MLM
41      APD
469     VMC
390     RCL
125     CZR
325    NCLH
143     DPZ
397      RL
331     NKE
377    POOL
73      BWA
400     ROL
180    FBHS
362    PENN
84      CCL
176       F
342     NVR
174    EXPE
145     DRI
370     PKG
369     PHM
408     SEE
169    ETSY
229     HSY
304     MKC
288      LW
363     PEP
Name: Symbol, dtype: object